In [1]:
from sneakers_ml.features.features import create_features
from sneakers_ml.features.resnet152 import get_resnet152_features

In [2]:
create_features(
    "data/training/brands-classification", "data/features/brands-classification", get_resnet152_features, "resnet"
)

  0%|          | 0/83 [00:00<?, ?it/s]

In [3]:
create_features(
    "data/training/brands-classification-splits",
    "data/features/brands-classification-splits",
    get_resnet152_features,
    "resnet",
)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]